### Reflection Pattern


In [5]:
import os
import re
import dotenv
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential, get_bearer_token_provider

dotenv.load_dotenv()

True

In [6]:
# Create a token provider
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(),
    "https://cognitiveservices.azure.com/.default"
)

# Get the access-token
token = token_provider()
print(f"Access-Token: {token}")

api_version = os.getenv("AZURE_OPENAI_API_VERSION")
api_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")

# Create the AzureOpenAI client
client = AzureOpenAIChatCompletionClient(
    azure_deployment="gpt-4o",
    model="gpt-4o",
    api_version=api_version,
    azure_endpoint=api_endpoint,
    azure_ad_token=token
)

Access-Token: eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IkpETmFfNGk0cjdGZ2lnTDNzSElsSTN4Vi1JVSIsImtpZCI6IkpETmFfNGk0cjdGZ2lnTDNzSElsSTN4Vi1JVSJ9.eyJhdWQiOiJodHRwczovL2NvZ25pdGl2ZXNlcnZpY2VzLmF6dXJlLmNvbSIsImlzcyI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0LzA4OTE1MjhmLWI3ODktNDFlZi05ZDEwLTg5NWE0NGY1ZjYyNC8iLCJpYXQiOjE3NDI1MzgwNzgsIm5iZiI6MTc0MjUzODA3OCwiZXhwIjoxNzQyNTQyMDgwLCJhY3IiOiIxIiwiYWlvIjoiQVVRQXUvOFpBQUFBMXB5RS9qeDZkUGJLL1ozNHF0NmM0YkxkZWZCREc2Q1lVaFdhazZnQXpZQ3NnUTBtMjRMVjNudVlTUVhIMlJPanlBeDd5NUpUWGNVL0lWc2Q4YWFZOXc9PSIsImFtciI6WyJwd2QiXSwiYXBwaWQiOiIwNGIwNzc5NS04ZGRiLTQ2MWEtYmJlZS0wMmY5ZTFiZjdiNDYiLCJhcHBpZGFjciI6IjAiLCJmYW1pbHlfbmFtZSI6Ik1pc2hyYSIsImdpdmVuX25hbWUiOiJTb3VyYXYiLCJncm91cHMiOlsiODA2ZWVkMzAtNTUzNC00MTlmLTg2NzgtOWYxMGQ2MmE1ZjRmIiwiMWVlOWMzNTEtMWYxNi00MzM0LWE4MjItZDZmZjY0OTYzNTUwIiwiNTI1ZWM5NTktYmQxYS00MWFiLTk5YmMtM2I4OTg4YTA2ZmVmIiwiMjQ3NTBlZTgtMjJlZC00ZjdmLWFiYWItMDVlOGY0NGRlMjQxIiwiZDQzM2E1MGEtNjBiZS00YmRkLWFkODYtNzlkYjYzY2Y3NzI1IiwiMDY4NGE2N2EtMjczZi00ODc3LWJlY2YtZDMxM

In [7]:
# Create the primary agent
Story_writer = AssistantAgent(
    name="Story_writer",
    model_client=client,
    system_message="""You are a helpful AI assistant that write technical blogposts. 
    You can help them with story building, sharing real facts and figures, examples, snippets and other useful links. 
    A code snippet should only be added if the story needs it. Keep the blogposts engaging and short within 300 words.
    """,
)

# Create the critic agent
Story_reviewer = AssistantAgent(
    name="Story_reviewer",
    model_client=client,
    system_message="""You are a helpful AI assistant which provides constructive feedback on the technical blogposts to make sure they stick to the topic, and provide relevant facts, figures, examples and correct code snippets that align to the topic.
    Make sure the blogs have great story-telling and check if the blog content needs a code snippet or not. Only if code snippet is needed, it should be added. 
    Respond with 'APPROVE' to when your feedbacks are addressed
    """,
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination('APPROVE')

# Create a team with the primary and critic agents.
team = RoundRobinGroupChat([Story_writer, Story_reviewer], termination_condition=text_termination)

In [ ]:
# Define the main asynchrounous function
async def main() -> None:
    # Stream the messages to the console
    output = await Console(
        team.run_stream(task="Write a blog post of Data Governance for AI")
    )

    print(f"Final Output: {output.messages[-2].content}")
    print(f"\nOutput_Stop_Reason: {output.stop_reason}")

    # Check if output contains stop_reason "APPROVE"
    if "APPROVE" in output.stop_reason:
        print("\nBlog post approved")
        blog_post_content = output.messages[-2].content

        if blog_post_content:
            print(f"\nBlog_Post_Contents: {blog_post_content}")
            print(f"\nBlog_Post_Length: {len(blog_post_content)}")

            # Extract the blog post title
            title_match = re.search(r'\*\*(.*?)\*\*', blog_post_content)
            if title_match:
                blog_post_title = title_match.group(1).split(':')[0].replace(' ', '_')    
                print(f"\nBlog_Post_Title: {blog_post_title}")
                filePath = os.path.join("outputs", f"{blog_post_title}.md")
                print(f"\nFile_Path: {filePath}")

                # Ensure the output directory exists
                os.makedirs("outputs", exist_ok=True)
                
                # Save the output to a .md file in ./outputs directory
                with open(filePath, "w", encoding="utf-8") as f:
                    f.write(f"# {title_match.group(1)}\n\n")
                    f.write(blog_post_content)
                print(f"\nFile created at: {filePath}")
            else:
                print("\nTitle not found in the blog post content.") 
        else:
            print("\nNo blog post content generated")
    else:
        print("\nBlog post not approved")
    
    

# Check if there's an existing event loop
try:
    loop = asyncio.get_running_loop()
except RuntimeError:
    loop = None

if loop and loop.is_running():
    # If there's an existing event loop, use it to run the main coroutine
    asyncio.ensure_future(main())
else:
    # Otherwise, use asyncio.run()
    asyncio.run(main())

---------- user ----------
Write a blog post of Data Governance for AI
---------- Story_writer ----------
**The Vital Role of Data Governance in AI Deployment**  

In today’s data-driven world, Artificial Intelligence (AI) has become a cornerstone of innovation across industries. However, without proper data governance, even the most sophisticated AI models can falter. Data governance in AI refers to establishing policies, procedures, and frameworks for managing data quality, privacy, security, and ethics throughout its lifecycle.  

### Why Does Data Governance Matter for AI?  

AI thrives on good data. A lack of governance can lead to biased, incomplete, or inaccurate datasets, which in turn result in flawed AI outputs. For instance, consider an AI model designed for hiring decisions. If trained on biased historical data without oversight, the model risks perpetuating systemic inequalities.  

Good governance ensures that an organization’s data isn’t just plentiful but also meaningfu